In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from lmmvae.dim_reduction import run_dim_reduction
from lmmvae.simulation import Count
from lmmvae.utils import get_dummies, get_posterior_b_root

In [3]:
census = pd.read_csv('../../data/uscensus_df.csv')

In [4]:
print(census.shape)
census.head()

(71371, 34)


,total_pop,men,women,hispanic,white,black,native,asian,pacific,voting_age_citizen,...,mean_commute,employed,private_work,public_work,self_employed,family_work,unemployment,lat,long,location_id
0,7.520235,0.487263,0.512737,0.024,0.863,0.052,0.000,0.012,0.000,0.762602,...,0.245,0.477507,0.742,0.212,0.045,0.000,0.046,-3.8586,3.272735,424
1,7.683404,0.537293,0.462707,0.011,0.416,0.545,0.000,0.010,0.000,0.760589,...,0.222,0.392265,0.759,0.150,0.090,0.000,0.034,-3.8586,3.272735,424
2,8.127109,0.452880,0.547120,0.080,0.614,0.265,0.006,0.007,0.004,0.732644,...,0.231,0.437814,0.733,0.211,0.048,0.007,0.047,-3.8586,3.272735,424
3,8.358666,0.468948,0.531052,0.096,0.803,0.071,0.005,0.002,0.000,0.763300,...,0.259,0.433326,0.758,0.197,0.045,0.000,0.061,-3.8586,3.272735,424
4,9.206834,0.507175,0.492825,0.009,0.775,0.164,0.000,0.031,0.000,0.725439,...,0.210,0.480381,0.714,0.241,0.045,0.000,0.023,-3.8586,3.272735,424


In [5]:
# computing the kernel matrix once
coords = census.groupby(['location_id','lat', 'long']).size().index.to_frame().values
coords[:5]

array([[  0.        , -10.        ,   5.23002823],
       [  1.        ,  -9.74535322,   5.42403737],
       [  2.        ,  -9.32361758,   5.14619821],
       [  3.        ,  -9.3042758 ,  -0.5750098 ],
       [  4.        ,  -9.2882231 ,   5.45063069]])

In [6]:
dist_matrix = squareform(pdist(coords[:,1:])) ** 2
dist_matrix.shape

(3108, 3108)

In [7]:
# spatial features name change
census.rename({'lat': 'D1', 'long': 'D2', 'location_id': 'z0'}, axis=1, inplace=True)

In [8]:
X = census.drop('income', axis=1)

In [9]:
# special features: t for longitudinal, D1 and D2 for spatial (longitude, latitude)
new_cols = [col for col in X.columns if col not in ['D1', 'D2', 'z0']] + ['D1', 'D2', 'z0']
X = X[new_cols]
X.columns

Index(['total_pop', 'men', 'women', 'hispanic', 'white', 'black', 'native',
       'asian', 'pacific', 'voting_age_citizen', 'poverty', 'child_poverty',
       'professional', 'service', 'office', 'construction', 'production',
       'drive', 'carpool', 'transit', 'walk', 'other_transp', 'work_at_home',
       'mean_commute', 'employed', 'private_work', 'public_work',
       'self_employed', 'family_work', 'unemployment', 'D1', 'D2', 'z0'],
      dtype='object')

In [10]:
# params for LMMVAE and other methods, some unnecessary for current use-case therefore are none
mode = 'spatial2'
n_sig2bs = 0 # no categorical features
n_sig2bs_spatial = 2
n_neurons = [1000, 500]
dropout = None
activation = 'relu'
RE_cols_prefix = 'z'
thresh = None
epochs = 200
qs = []
q_spatial = len(X['z0'].unique())
batch_size = 1000
patience = None
kernel = np.exp(-dist_matrix / (2 * 1))
Z = get_dummies(X['z0'], q_spatial)
kernel_root = get_posterior_b_root(kernel, Z, sig2e=1, n=X.shape[0], n_samp=10000)
U = None
B_list = None
est_cors = []
n_neurons_re = n_neurons
max_spatial_locs = 100
time2measure_dict = None
pred_unknown_clusters = False # Change for Unknown mode

In [11]:
res = pd.DataFrame(columns=['d', 'beta', 're_prior', 'experiment', 'exp_type', 'mse_X', 'sigma_b0_spatial_est', 'n_epoch', 'time'])
kf = KFold(n_splits=5, shuffle=True, random_state=40)
counter = Count().gen()
x_cols = [col for col in X.columns if col not in ['z0']]
x_cols_to_scale = [col for col in x_cols if col not in ['D1', 'D2']]

In [12]:
def iterate_reg_types(X_train, X_test, counter, d, beta, re_prior, i, verbose):
    mse_svgpvae_16_2, _, _, n_epochs_svgpvae_16_2, time_svgpvae_16_2 = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'svgpvae-10-16-2',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished svgpvae_16_2, mse: %.3f' % mse_svgpvae_16_2)
    mse_lmmvae, sigmas, _, n_epochs_lmmvae, time_lmmvae = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'lmmvae',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished lmmvae, mse: %.3f' % mse_lmmvae)
    mse_lmmvae_sfc, sigmas_sfc, _, n_epochs_lmmvae_sfc, time_lmmvae_sfc = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'lmmvae-sfc',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished lmmvae-sfc, mse: %.3f' % mse_lmmvae_sfc)
    mse_ig, _, _, n_epochs_ig, time_ig = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'pca-ignore',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished pca-ignore, mse: %.3f' % mse_ig)
    mse_ohe, _, _, n_epochs_ohe, time_ohe = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'pca-ohe',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished pca-ohe, mse: %.3f' % mse_ohe)
    mse_vaeig, _, _, n_epochs_vaeig, time_vaeig = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'vae-ignore',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished vae-ignore, mse: %.3f' % mse_vaeig)
    mse_vaeohe, _, _, n_epochs_vaeohe, time_vaeohe = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'vae-ohe',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished vae-ohe, mse: %.3f' % mse_vaeohe)
    mse_vaeem, _, _, n_epochs_vaeem, time_vaeem = run_dim_reduction(X_train, X_test, x_cols, RE_cols_prefix, d, 'vae-embed',
            thresh, epochs, qs, q_spatial, n_sig2bs, n_sig2bs_spatial, est_cors, batch_size, patience, n_neurons, n_neurons_re, dropout,
            activation, mode, beta, re_prior, kernel_root, pred_unknown_clusters, max_spatial_locs, time2measure_dict, verbose, U, B_list)
    print('   finished vae-embed, mse: %.3f' % mse_vaeem)
    res.loc[next(counter)] = [d, beta, re_prior, i, 'svgpvae_16_2', mse_svgpvae_16_2, np.nan, n_epochs_svgpvae_16_2, time_svgpvae_16_2]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'lmmvae', mse_lmmvae, sigmas[2][0], n_epochs_lmmvae, time_lmmvae]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'lmmvae-sfc', mse_lmmvae_sfc, sigmas_sfc[2][0], n_epochs_lmmvae_sfc, time_lmmvae_sfc]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'pca-ignore', mse_ig, np.nan, n_epochs_ig, time_ig]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'pca-ohe', mse_ohe, np.nan, n_epochs_ohe, time_ohe]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'vae-ignore', mse_vaeig, np.nan, n_epochs_vaeig, time_vaeig]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'vae-ohe', mse_vaeohe, np.nan, n_epochs_vaeohe, time_vaeohe]
    res.loc[next(counter)] = [d, beta, re_prior, i, 'vae-em', mse_vaeem, np.nan, n_epochs_vaeem, time_vaeem]
    res.to_csv('res_us_income_random.csv')

In [13]:
betas = [0.001]
ds = [1, 2, 5]
re_priors = [0.001]

In [ ]:
# Unknown mode
if pred_unknown_clusters:
  for beta in betas:
    for d in ds:
      for re_prior in re_priors:
        print(f'beta: {beta}, d: {d}, re_prior: {re_prior}:')
        cluster_q = q_spatial
        for i, (train_clusters, test_clusters) in enumerate(kf.split(range(cluster_q))):
          print('  iteration %d' % i)
          X_train, X_test = X[X['z0'].isin(train_clusters)].copy(), X[X['z0'].isin(test_clusters)].copy()
          iterate_reg_types(X_train, X_test, counter, d, beta, re_prior, i, verbose=False)

In [15]:
# Random mode
for beta in betas:
  for d in ds:
    for re_prior in re_priors:
      print(f'beta: {beta}, d: {d}, re_prior: {re_prior}:')
      for i, (train_index, test_index) in enumerate(kf.split(X)):
        print('  iteration %d' % i)
        X_train, X_test = X.loc[train_index].copy(), X.loc[test_index].copy()
        iterate_reg_types(X_train, X_test, counter, d, beta, re_prior, i, verbose=False)